In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# https://data.census.gov/cedsci/
df = pd.read_csv(
    "ACSDT1Y2022.B08006_2024-09-08T001306/ACSDT1Y2022.B08006-Data.csv", header=1, index_col="Geography")

data = pd.DataFrame(index=df.index)

data["city"] = df["Geographic Area Name"]
data["city"] = data["city"].apply(lambda x: re.sub(r" [a-z]+(.+)?,", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r" [A-Z]+?,", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r"\/(.+?),", ",", x))
data["city"] = data["city"].apply(lambda x: re.sub(r"\-(.+?),", ",", x))

data["bicycle"] = df["Estimate!!Total:!!Bicycle"].values
# drop all rows where column bicycle is zero
data.drop(data[data["bicycle"] == 0].index, inplace=True)

print(data.to_string())

                                                    city  bicycle
Geography                                                        
1600000US0103076                         Auburn, Alabama      NaN
1600000US0107000                     Birmingham, Alabama    397.0
1600000US0121184                         Dothan, Alabama      NaN
1600000US0135896                         Hoover, Alabama      NaN
1600000US0150000                         Mobile, Alabama      NaN
1600000US0151000                     Montgomery, Alabama      NaN
1600000US0177256                     Tuscaloosa, Alabama      NaN
1600000US0203000                       Anchorage, Alaska    625.0
1600000US0404720                       Avondale, Arizona      NaN
1600000US0407940                        Buckeye, Arizona      NaN
1600000US0410670                   Casas Adobes, Arizona      NaN
1600000US0412000                       Chandler, Arizona    232.0
1600000US0423620                      Flagstaff, Arizona    546.0
1600000US0

In [3]:
df = pd.read_csv(
    "ACSST1Y2022.S0101_2024-09-08T001234/ACSST1Y2022.S0101-Data.csv", header=1, index_col="Geography")

data["population"] = df.loc[data.index,
                            "Estimate!!Total!!Total population"]
data["population"] = pd.to_numeric(data["population"], errors="coerce")
data.drop(data[data["population"] < 50_000].index, inplace=True)
print(data.to_string())

                                                    city  bicycle  population
Geography                                                                    
1600000US0103076                         Auburn, Alabama      NaN     80009.0
1600000US0107000                     Birmingham, Alabama    397.0    196353.0
1600000US0121184                         Dothan, Alabama      NaN     70524.0
1600000US0135896                         Hoover, Alabama      NaN     92427.0
1600000US0150000                         Mobile, Alabama      NaN    183282.0
1600000US0151000                     Montgomery, Alabama      NaN    196986.0
1600000US0177256                     Tuscaloosa, Alabama      NaN    110598.0
1600000US0203000                       Anchorage, Alaska    625.0    287145.0
1600000US0404720                       Avondale, Arizona      NaN     91618.0
1600000US0407940                        Buckeye, Arizona      NaN    105567.0
1600000US0410670                   Casas Adobes, Arizona      Na

In [4]:
data["per_100k"] = data["bicycle"] / data["population"] * 100_000
data.dropna(inplace=True)

chicago = data.loc["1600000US1714000", "per_100k"]

data["score"] = (data["per_100k"]/chicago).apply(np.log2)
data.sort_values(by="score", ascending=False, inplace=True)

In [5]:
print(data.to_string())
data.to_csv("biking.csv")

                                                    city  bicycle  population     per_100k     score
Geography                                                                                           
1600000US0807850                       Boulder, Colorado   5169.0    105482.0  4900.362147  2.735543
1600000US2562535               Somerville, Massachusetts   3706.0     79761.0  4646.381063  2.658762
1600000US2511000                Cambridge, Massachusetts   5012.0    118489.0  4229.928517  2.523287
1600000US1245025                    Miami Beach, Florida   2864.0     80027.0  3578.792158  2.282127
1600000US0649670               Mountain View, California   2483.0     81058.0  3063.238669  2.057712
1600000US0827425                  Fort Collins, Colorado   4796.0    169248.0  2833.711477  1.945347
1600000US4123850                          Eugene, Oregon   4877.0    177930.0  2740.965548  1.897338
1600000US3050200                       Missoula, Montana   1876.0     76960.0  2437.629938 